In [1]:
import json

with open("../data/graph_data/disease_all.json", "r") as file:
    data = json.load(file)

len(data)

97103

In [2]:
list_diseases = []

for i in data:
    if i['data']['properties'].get('name') != None:
        list_diseases.append({
            'identifier': i['data']['properties'].get('identifier'),
            'name': i['data']['properties'].get('name'),
            'source': i['data']['properties'].get('source')
        })
len(list_diseases)

11711

In [3]:
import hashlib

def generate_id(disease):
    md5_hash = hashlib.md5()
    md5_hash.update(disease['name'].encode('utf-8'))
    return md5_hash.hexdigest()

In [4]:
from langchain_huggingface import SentenceTransformerEmbeddings

embedding_model_node_retrieval = SentenceTransformerEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


def get_embeddings_disease_names_to_qdrant(list_diseases):
    points = []
    for disease in list_diseases:
        vector = embedding_model_node_retrieval.embed_documents([disease['name']])[0]

        point = {
            "id": generate_id(disease=disease),
            "vector": vector,
            "payload": {
                'identifier': disease['identifier'],
                'name': disease['name'],
                'source': disease['source']
            }
        }
        points.append(point)
    return points 

points = get_embeddings_disease_names_to_qdrant(list_diseases=list_diseases)
'done'

QDRANT_URL = https://db01a1a9-113c-4d41-8252-a545fee8a32f.europe-west3-0.gcp.cloud.qdrant.io
QDRANT_API_KEY = 4x-j-E2jkFlMLn7dasU7rh3yzy5V5vY3Ds9ctZuxd7O4-PiCqBtscQ


d:\Study\Công nghệ tri thức nâng cao\Final_Project\API For RAG_v2\get_sentence_models.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model_node_retrieval = SentenceTransformerEmbeddings(model_name=SENTENCE_EMBEDDING_MODEL_FOR_NODE_RETRIEVAL)


'done'

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
QDRANT_API_KEY=''
QDRANT_URL=''

def load_qdrant(collection_name, vector_size = 384):
  
  client = QdrantClient(api_key=QDRANT_API_KEY, url=QDRANT_URL)
  distance_metric = rest.Distance.COSINE

  if not client.collection_exists(collection_name):
      client.create_collection(
          collection_name=collection_name,
          vectors_config=rest.VectorParams(
              size=vector_size,
              distance=distance_metric
          )
      )

  return client

qdrant_client = load_qdrant(collection_name = 'SPOKE_DISEASES', vector_size=384)


In [6]:

chunk_size=10
for i in range(0, len(points), chunk_size):
    chunk = points[i:i + chunk_size]
    try:
        qdrant_client.upsert(
            collection_name='STORED_DISEASES_SPOKE',
            points=chunk
        )
        print(f"Successfully saved {i + chunk_size} disease names to Qdrant.")
    except Exception as e:
        print(f"Error upserting disease names in Qdrant: {e}")

# qdrant_client.upsert(collection_name='STORED_DISEASES_SPOKE', points=points)

Successfully saved 10 disease names to Qdrant.
Successfully saved 20 disease names to Qdrant.
Successfully saved 30 disease names to Qdrant.
Successfully saved 40 disease names to Qdrant.
Successfully saved 50 disease names to Qdrant.
Successfully saved 60 disease names to Qdrant.
Successfully saved 70 disease names to Qdrant.
Successfully saved 80 disease names to Qdrant.
Successfully saved 90 disease names to Qdrant.
Successfully saved 100 disease names to Qdrant.
Successfully saved 110 disease names to Qdrant.
Successfully saved 120 disease names to Qdrant.
Successfully saved 130 disease names to Qdrant.
Successfully saved 140 disease names to Qdrant.
Successfully saved 150 disease names to Qdrant.
Successfully saved 160 disease names to Qdrant.
Successfully saved 170 disease names to Qdrant.
Successfully saved 180 disease names to Qdrant.
Successfully saved 190 disease names to Qdrant.
Successfully saved 200 disease names to Qdrant.
Successfully saved 210 disease names to Qdrant.
S

In [7]:
len(points)

11711